In [ ]:
import ee
import pandas as pd
import geopandas as gpd
import json
import geemap

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from parameters.config_output_naming import geo_id_column
from modules.agstack_to_gee import start_agstack_session, get_agstack_token, geometry_to_geo_id, geo_id_or_ids_to_feature_collection, register_feature_and_set_geo_id, register_fc_and_set_geo_id 
from parameters.config_asr_url_info import asset_registry_base,user_registry_base
from parameters.config_asr_credentials import email,password

def whisp_alt_stats_as_df (roi): 
    %store roi
    %run alt_whisp_stats.ipynb   
    return df_out

#fetch and convert into feature collection
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)


#### List of Geo_ids

In [ ]:
GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids

#make into a feature collection
roi = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#WHISP stats as a dataframe
whisp_alt_stats_as_df(roi)

#### Show map for drawing geometry

In [ ]:
Map = geemap.Map(center=[6,-2], zoom=12)
sc = Sidecar(title='draw polygon')
with sc:display(Map)

#### User input needed: draw boundary on map 

#### Fetch geo id and Whisp stats

In [ ]:
#teakse
drawn_feature = (Map.draw_last_feature)

#register geo id and add to feature
feature_w_geo_id = register_feature_and_set_geo_id(
    drawn_feature,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True)

roi = feature_w_geo_id

whisp_alt_stats_as_df(ee.FeatureCollection(roi))


#### Get Geo_id

In [ ]:
geometry = ee.Geometry.Polygon([
    [-122.091, 37.418],
    [-122.086, 37.422],
    [-122.079, 37.425],
    [-122.086, 37.418]
])

# Convert the polygon to WKT using the function
geo_id = geometry_to_geo_id(geometry,token,session,asset_registry_base,debug=True)

# Print the WKT representation
print('geo_id representation:', geo_id)

In [ ]:
# Define an example ee.Feature with a polygon geometry
feature = ee.Feature(ee.Geometry.Polygon([
    [-122.091, 37.418],
    [-122.086, 37.422],
    [-122.079, 37.425],
    [-122.086, 37.418]
]))

feature_w_properties = register_feature_and_set_geo_id(feature,geo_id_column,token,session,asset_registry_base,debug=True)

roi =ee.FeatureCollection(feature_w_properties)


#### From asset (feature collection) 

In [ ]:
example_plot_feature_col_asset = "projects/fdap-remi/assets/selected_segments_peru_vector"
roi = ee.FeatureCollection(example_plot_feature_col_asset)

whisp_alt_stats_as_df(roi)

#### Feature collection

In [ ]:
# geom_1 =ee.Geometry.Polygon(
#         [[[102.19867776108482, 3.0835809888660344],
#           [102.1988172359536, 3.082670357801012],
#           [102.20001886559227, 3.082078035157836],
#           [102.2013385124276, 3.081745922272887],
#           [102.2014355657636, 3.0825815609469625],
#           [102.20011591892828, 3.0836528916202233]]]);

# geom_2 = ee.Geometry.Polygon(
#         [[[102.18777620396249, 3.079459767638651],
#           [102.18831264576546, 3.0785384196951506],
#           [102.1895786484205, 3.0792026473624907],
#           [102.18899929127328, 3.08005971470826]]],None, False);

# feat_1 = ee.Feature(geom_1)
# feat_2 = ee.Feature(geom_2)

# feature_col = ee.FeatureCollection([feat_1,feat_2])

# roi = register_fc_and_set_geo_id(
#     feature_col,
#     geo_id_column,
#     token,
#     session,
#     asset_registry_base,
#     debug=True
# )
    
# roi = feature_col(roi)

# whisp_alt_stats_as_df(roi)


#### Shapefile polygon

In [ ]:
shp = "test_ceo_all.shp.zip"

# Load your geospatial dataset
gdf = gpd.read_file(shp)

geo_json = gdf.to_json()

roi = ee.FeatureCollection(json.loads(geo_json))

roi = roi.select([geo_id_column])

whisp_alt_stats_as_df(roi)

#### Points

In [ ]:
#get test points from polygon feature collection (agstack currently not working)

# Compute centroids of each polygon
def getCentroid(feature):
    keepProperties = [geo_id_column];
    # Get the centroid of the feature's geometry.
    centroid = feature.geometry().centroid();
    #Return a new Feature, copying properties from the old Feature.
    return ee.Feature(centroid).copyProperties(feature, keepProperties);

# roi = roi.map(getCentroid)

roi

whisp_alt_stats_as_df(roi)

#### Export CSV

In [ ]:
#export output csv
out_file_wide = "alt_whisp_output_table.csv"

df_out.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)